<a href="https://colab.research.google.com/github/kareemullah123456789/bigdatafoundation-july8-new/blob/main/Copy_of_Copy_of_PysparkDataFrame_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# ============================================================================
# SECTION 1: INSTALLATION AND SETUP FOR GOOGLE COLAB
# ============================================================================

# Install PySpark and required dependencies
!pip install pyspark findspark

# Import necessary libraries
import findspark
findspark.init()


In [2]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, avg, count, sum, max, min
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import pandas as pd


In [3]:

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("myfirstapp") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print("Spark Session Created Successfully!")
print(f"Spark Version: {spark.version}")


Spark Session Created Successfully!
Spark Version: 3.5.1


In [32]:
employees1 = spark.createDataFrame([(1,"ashi","IT",7000),(2,"kare","HR",40000),
                                    (3,"lattu","IT",9000),
                                    (4,"chutki","finance",30000)], ["id","name","dept","salary"])

In [36]:
from numpy import select
result1 = (employees1.filter("salary>7000")
        .select("name","dept","salary")
          .withColumn("bonus", employees1.salary * 0.1))

In [37]:
result1.show()

+------+-------+------+------+
|  name|   dept|salary| bonus|
+------+-------+------+------+
|  kare|     HR| 40000|4000.0|
| lattu|     IT|  9000| 900.0|
|chutki|finance| 30000|3000.0|
+------+-------+------+------+



In [39]:
result1.explain(True)

== Parsed Logical Plan ==
'Project [name#286, dept#287, salary#288L, (salary#288L * 0.1) AS bonus#321]
+- Project [name#286, dept#287, salary#288L]
   +- Filter (salary#288L > cast(7000 as bigint))
      +- LogicalRDD [id#285L, name#286, dept#287, salary#288L], false

== Analyzed Logical Plan ==
name: string, dept: string, salary: bigint, bonus: double
Project [name#286, dept#287, salary#288L, (cast(salary#288L as double) * 0.1) AS bonus#321]
+- Project [name#286, dept#287, salary#288L]
   +- Filter (salary#288L > cast(7000 as bigint))
      +- LogicalRDD [id#285L, name#286, dept#287, salary#288L], false

== Optimized Logical Plan ==
Project [name#286, dept#287, salary#288L, (cast(salary#288L as double) * 0.1) AS bonus#321]
+- Filter (isnotnull(salary#288L) AND (salary#288L > 7000))
   +- LogicalRDD [id#285L, name#286, dept#287, salary#288L], false

== Physical Plan ==
*(1) Project [name#286, dept#287, salary#288L, (cast(salary#288L as double) * 0.1) AS bonus#321]
+- *(1) Filter (isnot

In [10]:
df = spark.createDataFrame([
    (2, "Alice"), (5, "Bob")], schema=["age", "name"])
df

DataFrame[age: bigint, name: string]

In [ ]:
# transformation
#select , filter , groupby , join , withColumn, createDataFrame, distinct , orderby , limit , drop , union

In [12]:
#actions
# show, collect , count , take , first , write.csv , write.parquet ,

In [11]:
df.show()

+---+-----+
|age| name|
+---+-----+
|  2|Alice|
|  5|  Bob|
+---+-----+



In [27]:
people = spark.createDataFrame([
    {"deptId": 1, "age": 40, "name": "Hyukjin Kwon", "gender": "M", "salary": 50},
    {"deptId": 1, "age": 50, "name": "Takuya Ueshin", "gender": "M", "salary": 100},
    {"deptId": 2, "age": 60, "name": "Xinrong Meng", "gender": "F", "salary": 150},
    {"deptId": 3, "age": 20, "name": "Haejoon Lee", "gender": "M", "salary": 200}
])

In [28]:
people.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[age#269L,deptId#270L,gender#271,name#272,salary#273L]




In [16]:
people.show()

+---+------+------+-------------+------+
|age|deptId|gender|         name|salary|
+---+------+------+-------------+------+
| 40|     1|     M| Hyukjin Kwon|    50|
| 50|     1|     M|Takuya Ueshin|   100|
| 60|     2|     F| Xinrong Meng|   150|
| 20|     3|     M|  Haejoon Lee|   200|
+---+------+------+-------------+------+



In [29]:
result = people.filter(people.age > 40).select("name", "salary").withColumn("bonus", people.salary * 0.10)

In [30]:
result.explain()

== Physical Plan ==
*(1) Project [name#272, salary#273L, (cast(salary#273L as double) * 0.1) AS bonus#281]
+- *(1) Filter (isnotnull(age#269L) AND (age#269L > 40))
   +- *(1) Scan ExistingRDD[age#269L,deptId#270L,gender#271,name#272,salary#273L]




In [31]:
result.explain(True)

== Parsed Logical Plan ==
'Project [name#272, salary#273L, (salary#273L * 0.1) AS bonus#281]
+- Project [name#272, salary#273L]
   +- Filter (age#269L > cast(40 as bigint))
      +- LogicalRDD [age#269L, deptId#270L, gender#271, name#272, salary#273L], false

== Analyzed Logical Plan ==
name: string, salary: bigint, bonus: double
Project [name#272, salary#273L, (cast(salary#273L as double) * 0.1) AS bonus#281]
+- Project [name#272, salary#273L]
   +- Filter (age#269L > cast(40 as bigint))
      +- LogicalRDD [age#269L, deptId#270L, gender#271, name#272, salary#273L], false

== Optimized Logical Plan ==
Project [name#272, salary#273L, (cast(salary#273L as double) * 0.1) AS bonus#281]
+- Filter (isnotnull(age#269L) AND (age#269L > 40))
   +- LogicalRDD [age#269L, deptId#270L, gender#271, name#272, salary#273L], false

== Physical Plan ==
*(1) Project [name#272, salary#273L, (cast(salary#273L as double) * 0.1) AS bonus#281]
+- *(1) Filter (isnotnull(age#269L) AND (age#269L > 40))
   +- *(

In [21]:
result.show()

+-------------+------+-----+
|         name|salary|bonus|
+-------------+------+-----+
|Takuya Ueshin|   100| 10.0|
| Xinrong Meng|   150| 15.0|
+-------------+------+-----+



In [22]:
result.count()

2

In [25]:
result.collect()

[Row(name='Takuya Ueshin', salary=100, bonus=10.0),
 Row(name='Xinrong Meng', salary=150, bonus=15.0)]

In [26]:
result.write.csv("/content/result")

In [ ]:
# transformation : make a plan do not exectute
#actions : execute the plan

In [7]:
# To create DataFrame using SparkSession
department = spark.createDataFrame([
    {"id": 1, "name": "PySpark"},
    {"id": 2, "name": "ML"},
    {"id": 3, "name": "Spark SQL"}
])

In [8]:
department.show()

+---+---------+
| id|     name|
+---+---------+
|  1|  PySpark|
|  2|       ML|
|  3|Spark SQL|
+---+---------+



In [9]:
people.show()

+---+------+------+-------------+------+
|age|deptId|gender|         name|salary|
+---+------+------+-------------+------+
| 40|     1|     M| Hyukjin Kwon|    50|
| 50|     1|     M|Takuya Ueshin|   100|
| 60|     2|     F| Xinrong Meng|   150|
| 20|     3|     M|  Haejoon Lee|   200|
+---+------+------+-------------+------+



In [ ]:
transform= people.filter(people.age > 30).join(
    department, people.deptId == department.id).groupBy(
    department.name, "gender").agg(
        {"salary": "avg", "age": "max"}).sort("max(age)")







In [ ]:
transform

DataFrame[name: string, gender: string, avg(salary): double, max(age): bigint]

In [ ]:
transform.show()

+-------+------+-----------+--------+
|   name|gender|avg(salary)|max(age)|
+-------+------+-----------+--------+
|PySpark|     M|       75.0|      50|
|     ML|     F|      150.0|      60|
+-------+------+-----------+--------+



In [41]:


# ============================================================================
# SECTION 2: CREATING DATAFRAMES
# ============================================================================

# Method 1: Create DataFrame from Python list
print("\n=== Creating DataFrame from List ===")
data = [
    ("Alice", 25, "Engineer", 75000),
    ("Bob", 30, "Manager", 85000),
    ("Charlie", 35, "Analyst", 65000),
    ("Diana", 28, "Designer", 70000),
    ("Eve", 32, "Engineer", 80000)
]

columns = ["Name", "Age", "Job", "Salary"]
df = spark.createDataFrame(data, columns)
df



=== Creating DataFrame from List ===


DataFrame[Name: string, Age: bigint, Job: string, Salary: bigint]

In [ ]:
df.show()  # action

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [42]:

# Method 2: Create DataFrame with explicit schema
print("\n=== Creating DataFrame with Schema ===")
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Job", StringType(), True),
    StructField("Salary", FloatType(), True)
])
columns = ["Name", "Age", "Job", "Salary"]

df_schema = spark.createDataFrame(data, schema=columns)
df_schema.printSchema()



=== Creating DataFrame with Schema ===
root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Job: string (nullable = true)
 |-- Salary: long (nullable = true)



In [43]:
df_schema.show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [44]:

# Method 3: Create DataFrame from Pandas DataFrame
print("\n=== Creating DataFrame from Pandas ===")
pandas_df = pd.DataFrame({
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones'],
    'Price': [999.99, 25.50, 75.00, 299.99, 150.00],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Audio'],
    'Stock': [50, 200, 150, 30, 75]
})
pandas_df


=== Creating DataFrame from Pandas ===


,Product,Price,Category,Stock
0,Laptop,999.99,Electronics,50
1,Mouse,25.50,Accessories,200
2,Keyboard,75.00,Accessories,150
3,Monitor,299.99,Electronics,30
4,Headphones,150.00,Audio,75


In [45]:

products_df = spark.createDataFrame(pandas_df)
products_df


DataFrame[Product: string, Price: double, Category: string, Stock: bigint]

In [46]:
products_df.show()

+----------+------+-----------+-----+
|   Product| Price|   Category|Stock|
+----------+------+-----------+-----+
|    Laptop|999.99|Electronics|   50|
|     Mouse|  25.5|Accessories|  200|
|  Keyboard|  75.0|Accessories|  150|
|   Monitor|299.99|Electronics|   30|
|Headphones| 150.0|      Audio|   75|
+----------+------+-----------+-----+



In [47]:

# ============================================================================
# SECTION 3: BASIC DATAFRAME OPERATIONS
# ============================================================================

print("\n=== Basic DataFrame Info ===")
print(f"Number of rows: {df.count()}")
print(f"Number of columns: {len(df.columns)}")
print(f"Column names: {df.columns}")



=== Basic DataFrame Info ===
Number of rows: 5
Number of columns: 4
Column names: ['Name', 'Age', 'Job', 'Salary']


In [ ]:
# df.Name
# df["Name"]

In [ ]:
df.show()


+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [48]:
df.select('*').show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [ ]:

# Select specific columns
print("\n=== Selecting Columns ===")
df.select("Name", "Job").show()



=== Selecting Columns ===
+-------+--------+
|   Name|     Job|
+-------+--------+
|  Alice|Engineer|
|    Bob| Manager|
|Charlie| Analyst|
|  Diana|Designer|
|    Eve|Engineer|
+-------+--------+



In [ ]:
df.show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [53]:
df.select(df.Name,(df.Salary * 1.5).alias("Increased_Salary")).show()

+-------+----------------+
|   Name|Increased_Salary|
+-------+----------------+
|  Alice|        112500.0|
|    Bob|        127500.0|
|Charlie|         97500.0|
|  Diana|        105000.0|
|    Eve|        120000.0|
+-------+----------------+



In [ ]:
df.show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [54]:
df.filter((df.Age >30) | (df.Salary > 70000)).show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [55]:
df.filter(df.Name.isin("Alice","Bob")).show()

+-----+---+--------+------+
| Name|Age|     Job|Salary|
+-----+---+--------+------+
|Alice| 25|Engineer| 75000|
|  Bob| 30| Manager| 85000|
+-----+---+--------+------+



In [56]:
df.filter(df.Name.like("B%")).show()

+----+---+-------+------+
|Name|Age|    Job|Salary|
+----+---+-------+------+
| Bob| 30|Manager| 85000|
+----+---+-------+------+



In [57]:
df.filter(df.Age.between(25,30)).show()

+-----+---+--------+------+
| Name|Age|     Job|Salary|
+-----+---+--------+------+
|Alice| 25|Engineer| 75000|
|  Bob| 30| Manager| 85000|
|Diana| 28|Designer| 70000|
+-----+---+--------+------+



In [58]:
people = spark.createDataFrame([
    {"deptId": 1, "age": 40, "name": "Hyukjin Kwon", "gender": "M", "salary": 50},
    {"deptId": 1, "age": 50, "name": "", "gender": "M", "salary": 100},
    {"deptId": 2, "age": 60, "name": "", "gender": "F", "salary": 150},
    {"deptId": 3, "age": 20, "name": "Haejoon Lee", "gender": "M", "salary": 200}
])

In [ ]:
people.show()

+---+------+------+------------+------+
|age|deptId|gender|        name|salary|
+---+------+------+------------+------+
| 40|     1|     M|Hyukjin Kwon|    50|
| 50|     1|     M|            |   100|
| 60|     2|     F|            |   150|
| 20|     3|     M| Haejoon Lee|   200|
+---+------+------+------------+------+



In [59]:
df1 = spark.createDataFrame([
    (10, 80.5, "Alice", None),
    (5, None, "Bob", None),
    (None, None, "Tom", None),
    (None, None, None, True)],
    schema=["age", "height", "name", "bool"])

In [ ]:
df1.show()

+----+------+-----+----+
| age|height| name|bool|
+----+------+-----+----+
|  10|  80.5|Alice|NULL|
|   5|  NULL|  Bob|NULL|
|NULL|  NULL|  Tom|NULL|
|NULL|  NULL| NULL|true|
+----+------+-----+----+



In [60]:
df.na.fill(0).show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [62]:

# Filter operations
print("\n=== Filtering Data ===")
df.filter(col("Age") > 30).show()



=== Filtering Data ===
+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|Charlie| 35| Analyst| 65000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [ ]:
df.filter(df.Age > 30).show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|Charlie| 35| Analyst| 65000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [ ]:

df.filter(df.Salary > 70000).show()


+-----+---+--------+------+
| Name|Age|     Job|Salary|
+-----+---+--------+------+
|Alice| 25|Engineer| 75000|
|  Bob| 30| Manager| 85000|
|  Eve| 32|Engineer| 80000|
+-----+---+--------+------+



In [ ]:

# Multiple conditions
print("\n=== Multiple Conditions ===")
df.filter((col("Age") > 25) & (col("Salary") > 70000)).show()



=== Multiple Conditions ===
+----+---+--------+------+
|Name|Age|     Job|Salary|
+----+---+--------+------+
| Bob| 30| Manager| 85000|
| Eve| 32|Engineer| 80000|
+----+---+--------+------+



In [ ]:

# Add new columns
print("\n=== Adding New Columns ===")
df_with_bonus = df.withColumn("Bonus", col("Salary") * 0.1)
df_with_bonus.show()




=== Adding New Columns ===
+-------+---+--------+------+------+
|   Name|Age|     Job|Salary| Bonus|
+-------+---+--------+------+------+
|  Alice| 25|Engineer| 75000|7500.0|
|    Bob| 30| Manager| 85000|8500.0|
|Charlie| 35| Analyst| 65000|6500.0|
|  Diana| 28|Designer| 70000|7000.0|
|    Eve| 32|Engineer| 80000|8000.0|
+-------+---+--------+------+------+



In [63]:


# Conditional columns
print("\n=== Conditional Columns ===")
df_with_category = df.withColumn(
    "Salary_Category",
    when(col("Salary") > 80000, "High")
    .when(col("Salary") > 70000, "Medium")
    .otherwise("Low")
)
df_with_category.show()




=== Conditional Columns ===
+-------+---+--------+------+---------------+
|   Name|Age|     Job|Salary|Salary_Category|
+-------+---+--------+------+---------------+
|  Alice| 25|Engineer| 75000|         Medium|
|    Bob| 30| Manager| 85000|           High|
|Charlie| 35| Analyst| 65000|            Low|
|  Diana| 28|Designer| 70000|            Low|
|    Eve| 32|Engineer| 80000|         Medium|
+-------+---+--------+------+---------------+



In [ ]:
df.withColumns({"bonus":df.Salary *0.1,"deduction":df.Salary *0.05}).show()

+-------+---+--------+------+------+---------+
|   Name|Age|     Job|Salary| bonus|deduction|
+-------+---+--------+------+------+---------+
|  Alice| 25|Engineer| 75000|7500.0|   3750.0|
|    Bob| 30| Manager| 85000|8500.0|   4250.0|
|Charlie| 35| Analyst| 65000|6500.0|   3250.0|
|  Diana| 28|Designer| 70000|7000.0|   3500.0|
|    Eve| 32|Engineer| 80000|8000.0|   4000.0|
+-------+---+--------+------+------+---------+



In [67]:
df.show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [76]:
# number of cores
import os
os.cpu_count()

2

In [77]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:              6
    Model:                   79
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4399.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

In [68]:
df.rdd.getNumPartitions()

2

In [69]:
df.rdd.glom().collect()

[[Row(Name='Alice', Age=25, Job='Engineer', Salary=75000),
  Row(Name='Bob', Age=30, Job='Manager', Salary=85000)],
 [Row(Name='Charlie', Age=35, Job='Analyst', Salary=65000),
  Row(Name='Diana', Age=28, Job='Designer', Salary=70000),
  Row(Name='Eve', Age=32, Job='Engineer', Salary=80000)]]

In [70]:
df.rdd.mapPartitionsWithIndex(lambda idx, it: [(idx, x) for x in it]).collect()

[(0, Row(Name='Alice', Age=25, Job='Engineer', Salary=75000)),
 (0, Row(Name='Bob', Age=30, Job='Manager', Salary=85000)),
 (1, Row(Name='Charlie', Age=35, Job='Analyst', Salary=65000)),
 (1, Row(Name='Diana', Age=28, Job='Designer', Salary=70000)),
 (1, Row(Name='Eve', Age=32, Job='Engineer', Salary=80000))]

In [71]:
df2 = df.repartition(4)

In [72]:
df2.rdd.glom().collect()

[[],
 [Row(Name='Bob', Age=30, Job='Manager', Salary=85000),
  Row(Name='Charlie', Age=35, Job='Analyst', Salary=65000)],
 [Row(Name='Alice', Age=25, Job='Engineer', Salary=75000),
  Row(Name='Diana', Age=28, Job='Designer', Salary=70000)],
 [Row(Name='Eve', Age=32, Job='Engineer', Salary=80000)]]

In [73]:
df2.rdd.mapPartitionsWithIndex(lambda idx, it: [(idx, x) for x in it]).collect()


[(1, Row(Name='Bob', Age=30, Job='Manager', Salary=85000)),
 (1, Row(Name='Charlie', Age=35, Job='Analyst', Salary=65000)),
 (2, Row(Name='Alice', Age=25, Job='Engineer', Salary=75000)),
 (2, Row(Name='Diana', Age=28, Job='Designer', Salary=70000)),
 (3, Row(Name='Eve', Age=32, Job='Engineer', Salary=80000))]

In [75]:
df2.rdd.getNumPartitions()

4

In [74]:
df3 = df.coalesce(1)
df3.rdd.getNumPartitions()

1

In [66]:


# ============================================================================
# SECTION 4: AGGREGATIONS AND GROUPING
# ============================================================================

print("\n=== Basic Aggregations ===")
df.agg(
    avg("Age").alias("Average_Age"),
    avg("Salary").alias("Average_Salary"),
    count("*").alias("Total_Records")
).explain(True)



=== Basic Aggregations ===
== Parsed Logical Plan ==
'Aggregate [avg('Age) AS Average_Age#693, avg('Salary) AS Average_Salary#695, count(1) AS Total_Records#697L]
+- LogicalRDD [Name#343, Age#344L, Job#345, Salary#346L], false

== Analyzed Logical Plan ==
Average_Age: double, Average_Salary: double, Total_Records: bigint
Aggregate [avg(Age#344L) AS Average_Age#693, avg(Salary#346L) AS Average_Salary#695, count(1) AS Total_Records#697L]
+- LogicalRDD [Name#343, Age#344L, Job#345, Salary#346L], false

== Optimized Logical Plan ==
Aggregate [avg(Age#344L) AS Average_Age#693, avg(Salary#346L) AS Average_Salary#695, count(1) AS Total_Records#697L]
+- Project [Age#344L, Salary#346L]
   +- LogicalRDD [Name#343, Age#344L, Job#345, Salary#346L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[], functions=[avg(Age#344L), avg(Salary#346L), count(1)], output=[Average_Age#693, Average_Salary#695, Total_Records#697L])
   +- Exchange SinglePartition, ENSURE_REQ

In [ ]:
(df.agg(
    avg("Age"),
    avg("Salary").alias("Average_Salary"),
    count("*").alias("Total_Records")
).show()

+--------+--------------+-------------+
|avg(Age)|Average_Salary|Total_Records|
+--------+--------------+-------------+
|    30.0|       75000.0|            5|
+--------+--------------+-------------+



In [ ]:

print("\n=== Group By Operations ===")
df.groupBy("Job").agg(
    count("*").alias("Count"),
    avg("Salary").alias("Avg_Salary"),
    max("Age").alias("Max_Age")
).show()



=== Group By Operations ===
+--------+-----+----------+-------+
|     Job|Count|Avg_Salary|Max_Age|
+--------+-----+----------+-------+
|Engineer|    2|   77500.0|     32|
| Manager|    1|   85000.0|     30|
|Designer|    1|   70000.0|     28|
| Analyst|    1|   65000.0|     35|
+--------+-----+----------+-------+



In [ ]:

# ============================================================================
# SECTION 5: DATA MANIPULATION
# ============================================================================

print("\n=== Sorting Data ===")
df.orderBy(df.Salary.desc()).show()



=== Sorting Data ===
+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|    Bob| 30| Manager| 85000|
|    Eve| 32|Engineer| 80000|
|  Alice| 25|Engineer| 75000|
|  Diana| 28|Designer| 70000|
|Charlie| 35| Analyst| 65000|
+-------+---+--------+------+



In [ ]:

# Rename columns
print("\n=== Renaming Columns ===")
df_renamed = df.withColumnRenamed("Job", "Position")
df_renamed.show()



=== Renaming Columns ===
+-------+---+--------+------+
|   Name|Age|Position|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [ ]:

# Drop columns
print("\n=== Dropping Columns ===")
df_dropped = df.drop("Age")
df_dropped.show()



=== Dropping Columns ===
+-------+--------+------+
|   Name|     Job|Salary|
+-------+--------+------+
|  Alice|Engineer| 75000|
|    Bob| Manager| 85000|
|Charlie| Analyst| 65000|
|  Diana|Designer| 70000|
|    Eve|Engineer| 80000|
+-------+--------+------+



In [ ]:

# Handle missing values
print("\n=== Handling Missing Values ===")
# Create data with nulls for demonstration
data_with_nulls = [
    ("John", 25, "Engineer", 75000),
    ("Jane", None, "Manager", 85000),
    ("Mike", 35, None, 65000),
    ("Sarah", 28, "Designer", None)
]

df_nulls = spark.createDataFrame(data_with_nulls, columns)
print("Data with nulls:")
df_nulls.show()



=== Handling Missing Values ===
Data with nulls:
+-----+----+--------+------+
| Name| Age|     Job|Salary|
+-----+----+--------+------+
| John|  25|Engineer| 75000|
| Jane|NULL| Manager| 85000|
| Mike|  35|    NULL| 65000|
|Sarah|  28|Designer|  NULL|
+-----+----+--------+------+



In [ ]:

# Fill nulls
df_filled = df_nulls.fillna({"Age": 0, "Job": "Unknown", "Salary": 0})
print("After filling nulls:")
df_filled.show()


After filling nulls:
+-----+---+--------+------+
| Name|Age|     Job|Salary|
+-----+---+--------+------+
| John| 25|Engineer| 75000|
| Jane|  0| Manager| 85000|
| Mike| 35| Unknown| 65000|
|Sarah| 28|Designer|     0|
+-----+---+--------+------+



In [ ]:

# Drop rows with nulls
df_no_nulls = df_nulls.dropna()
print("After dropping nulls:")
df_no_nulls.show()


After dropping nulls:
+----+---+--------+------+
|Name|Age|     Job|Salary|
+----+---+--------+------+
|John| 25|Engineer| 75000|
+----+---+--------+------+



In [ ]:
df.show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [ ]:

# ============================================================================
# SECTION 6: JOINS
# ============================================================================

print("\n=== Join Operations ===")
# Create department data
dept_data = [
    ("Engineer", "Technology"),
    ("Manager", "Administration"),
    ("Analyst", "Business"),
    ("Designer", "Creative")
]

dept_df = spark.createDataFrame(dept_data, ["Job", "Department"])
dept_df.show()



=== Join Operations ===
+--------+--------------+
|     Job|    Department|
+--------+--------------+
|Engineer|    Technology|
| Manager|Administration|
| Analyst|      Business|
|Designer|      Creative|
+--------+--------------+



In [ ]:
df.show()

+-------+---+--------+------+
|   Name|Age|     Job|Salary|
+-------+---+--------+------+
|  Alice| 25|Engineer| 75000|
|    Bob| 30| Manager| 85000|
|Charlie| 35| Analyst| 65000|
|  Diana| 28|Designer| 70000|
|    Eve| 32|Engineer| 80000|
+-------+---+--------+------+



In [ ]:

# Inner join
print("Inner Join:")
joined_df = df.join(dept_df, "Job", "inner")
joined_df.show()

Inner Join:
+--------+-------+---+------+--------------+
|     Job|   Name|Age|Salary|    Department|
+--------+-------+---+------+--------------+
| Analyst|Charlie| 35| 65000|      Business|
|Designer|  Diana| 28| 70000|      Creative|
|Engineer|  Alice| 25| 75000|    Technology|
|Engineer|    Eve| 32| 80000|    Technology|
| Manager|    Bob| 30| 85000|Administration|
+--------+-------+---+------+--------------+



In [ ]:


# Left join
print("Left Join:")
left_joined = df.join(dept_df, "Job", "left")
left_joined.show()


Left Join:
+--------+-------+---+------+--------------+
|     Job|   Name|Age|Salary|    Department|
+--------+-------+---+------+--------------+
|Engineer|  Alice| 25| 75000|    Technology|
| Manager|    Bob| 30| 85000|Administration|
|Designer|  Diana| 28| 70000|      Creative|
| Analyst|Charlie| 35| 65000|      Business|
|Engineer|    Eve| 32| 80000|    Technology|
+--------+-------+---+------+--------------+



In [ ]:


# ============================================================================
# SECTION 8: SQL QUERIES
# ============================================================================

print("\n=== SQL Queries ===")
# Register DataFrame as temporary view
df.createOrReplaceTempView("employees")

# Run SQL query
result = spark.sql("""
    SELECT Job, AVG(Salary) as avg_salary, COUNT(*) as count
    FROM employees
    GROUP BY Job
    ORDER BY avg_salary DESC
""")
result.show()

# ============================================================================
# SECTION 9: READING AND WRITING DATA
# ============================================================================



=== SQL Queries ===
+--------+----------+-----+
|     Job|avg_salary|count|
+--------+----------+-----+
| Manager|   85000.0|    1|
|Engineer|   77500.0|    2|
|Designer|   70000.0|    1|
| Analyst|   65000.0|    1|
+--------+----------+-----+



In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls /content/drive/MyDrive/cde_data/pyspark/

In [ ]:
#/content/sample_data
print("\n=== Writing Data ===")
# Write to CSV (this will create a directory with part files)
df.write.mode("overwrite").option("header", "true").csv("/content/drive/MyDrive/cde_data/pyspark/")



=== Writing Data ===


In [ ]:
df.coalesce(1).write.mode("overwrite").option("header", "true").csv("/content/drive/MyDrive/cde_data/pyspark/new")
#

In [ ]:
df.write.partitionBy("Age").mode("overwrite").option("header", "true").csv("/content/drive/MyDrive/cde_data/pyspark/partition")

In [ ]:

# Write to JSON
df.write.mode("overwrite").json("/content/drive/MyDrive/cde_data/pyspark/json")


In [ ]:

# Write to Parquet
df.write.mode("overwrite").parquet("/content/drive/MyDrive/cde_data/pyspark/employees_parquet")

print("Data written successfully!")


Data written successfully!


In [ ]:

# ============================================================================
# SECTION 10: PERFORMANCE OPTIMIZATION
# ============================================================================

print("\n=== Performance Tips ===")

# Cache DataFrame for repeated operations
df_cached = df.cache()
print("DataFrame cached")

# Check partitions
print(f"Number of partitions: {df.rdd.getNumPartitions()}")



=== Performance Tips ===
DataFrame cached
Number of partitions: 2


In [ ]:

# Repartition for better performance
df_repartitioned = df.repartition(3)
print(f"After repartitioning: {df_repartitioned.rdd.getNumPartitions()}")


After repartitioning: 3


In [ ]:

# ============================================================================
# SECTION 11: WORKING WITH DIFFERENT DATA TYPES
# ============================================================================

print("\n=== Working with Arrays and Maps ===")
from pyspark.sql.functions import array, map_from_arrays, explode



=== Working with Arrays and Maps ===


In [ ]:

# Create DataFrame with complex types
complex_data = [
    ("Alice", ["Python", "Spark", "SQL"], {"skill1": "Expert", "skill2": "Intermediate"}),
    ("Bob", ["Java", "Hadoop"], {"skill1": "Advanced", "skill2": "Expert"}),
    ("Charlie", ["R", "Statistics"], {"skill1": "Intermediate", "skill2": "Advanced"})
]

complex_df = spark.createDataFrame(complex_data, ["Name", "Skills", "SkillLevels"])
complex_df.show(truncate=False)


+-------+--------------------+--------------------------------------------+
|Name   |Skills              |SkillLevels                                 |
+-------+--------------------+--------------------------------------------+
|Alice  |[Python, Spark, SQL]|{skill1 -> Expert, skill2 -> Intermediate}  |
|Bob    |[Java, Hadoop]      |{skill1 -> Advanced, skill2 -> Expert}      |
|Charlie|[R, Statistics]     |{skill1 -> Intermediate, skill2 -> Advanced}|
+-------+--------------------+--------------------------------------------+



In [ ]:

# Explode array
exploded_df = complex_df.select("Name", explode("Skills").alias("Skill"))
exploded_df.show()


+-------+----------+
|   Name|     Skill|
+-------+----------+
|  Alice|    Python|
|  Alice|     Spark|
|  Alice|       SQL|
|    Bob|      Java|
|    Bob|    Hadoop|
|Charlie|         R|
|Charlie|Statistics|
+-------+----------+



In [ ]:

# ============================================================================
# SECTION 12: USER-DEFINED FUNCTIONS (UDF)
# ============================================================================

print("\n=== User Defined Functions ===")
from pyspark.sql.functions import udf

# Define a UDF
def salary_grade(salary):
    if salary > 80000:
        return "A"
    elif salary > 70000:
        return "B"
    else:
        return "C"

# Register UDF
salary_grade_udf = udf(salary_grade, StringType())



=== User Defined Functions ===


In [ ]:
salary_grade_udf

<function __main__.salary_grade(salary)>

In [ ]:

# Apply UDF
df_with_grade = df.withColumn("Grade", salary_grade_udf(col("Salary")))
df_with_grade.show()



+-------+---+--------+------+-----+
|   Name|Age|     Job|Salary|Grade|
+-------+---+--------+------+-----+
|  Alice| 25|Engineer| 75000|    B|
|    Bob| 30| Manager| 85000|    A|
|Charlie| 35| Analyst| 65000|    C|
|  Diana| 28|Designer| 70000|    C|
|    Eve| 32|Engineer| 80000|    B|
+-------+---+--------+------+-----+

